# Analyzing Subnetworks

This notebook provides an example of analyzing subnetworks for a study that uses drugs to target transcription factors.  We care about networks where proteins bind / inhibit / activate each other.

In [ ]:
!brew install graphviz

In [1]:
# Reference: https://pygraphviz.github.io/documentation/stable/install.html
!pip install -q pygraphviz \
    --config-settings=--global-option=build_ext \
    --config-settings=--global-option="-I$(brew --prefix graphviz)/include" \
    --config-settings=--global-option="-L$(brew --prefix graphviz)/lib"

In [ ]:
!pip install -q git+https://github.com/gyorilab/adeft.git

In [2]:
!pip install -q -r requirements.txt

## STEP 1: Import MSstats Dataset

First, we will import MSstats datasets as pandas dataframes.  The MSstats dataset consist of the output of the MSstats groupComparison function, which consists of protein p-values.

We filter the datasets to smaller sizes based on p-value.  You can adjust those parameters as well.

In [10]:
import pandas as pd

P_VALUE_LOGFC_PATH = "model.csv" # Set this path yourself
LABELS_FILTER = ["DMSO-PF477736"]
#LABELS_FILTER = ["DMSO-DbET6"]
#LABELS_FILTER = ["DMSO-Nuc"]
P_VALUE_FILTER = 0.05 # Adjust this yourself
MAIN_TARGETS = ["CHK1_HUMAN"]

def construct_pvalue_logfc_df(filename):
    """Return a filtered data frame from the given data file."""
    pandas_df = pd.read_csv(filename)
    pandas_df = pandas_df.loc[
        ((pandas_df['issue'].isnull()) & (pandas_df['adj.pvalue'] < P_VALUE_FILTER) 
         | (pandas_df['Protein'].isin(MAIN_TARGETS))) 
        & (pandas_df['Label'].isin(LABELS_FILTER))]
    
    return pandas_df

pvalue_logfc_df = construct_pvalue_logfc_df(P_VALUE_LOGFC_PATH)
pvalue_logfc_df

,Protein,Label,log2FC,SE,Tvalue,DF,pvalue,adj.pvalue,issue,MissingPercentage,ImputationPercentage
30606,CHK1_HUMAN,DMSO-PF477736,0.115541,0.157794,0.732229,238.0,4.647490e-01,9.980494e-01,NaN,0.334667,0.0
32541,CLH1_HUMAN,DMSO-PF477736,-0.269784,0.056036,-4.814489,260.0,2.506764e-06,3.094600e-03,NaN,0.143333,0.0
63546,FCGRN_HUMAN,DMSO-PF477736,-0.674851,0.132991,-5.074427,179.0,9.674775e-07,1.592468e-03,NaN,0.195918,0.0
115656,NECP2_HUMAN,DMSO-PF477736,0.693701,0.097077,7.145875,259.0,9.076739e-12,2.241047e-08,NaN,0.040000,0.0
152151,RFA1_HUMAN,DMSO-PF477736,-0.603824,0.076190,-7.925239,260.0,6.705747e-14,3.311298e-10,NaN,0.120667,0.0


## STEP 2: ID CONVERSION

INDRA Cogex only accepts HGNC IDs at the time of this writing.  However, the dataset provided in the example above contains uniprot mnemonic IDs.  

Luckily, INDRA has code to convert uniprot mnemonic IDs into HGNC ids. For now, we will store this mapping in a separate dictionary

In [11]:
from indra.databases import uniprot_client

def uniprot_to_hgnc_id(uniprot_mnemonic):
    """Get an HGNC ID from a UniProt mnemonic."""
    uniprot_id = uniprot_client.get_id_from_mnemonic(uniprot_mnemonic)
    if uniprot_id:
        return uniprot_client.get_hgnc_id(uniprot_id)
    else:
        return None

uniprot_to_hgnc_id("CLH1_HUMAN")

'2092'

In [12]:
def uniprot_to_hgnc_gene_name(uniprot_mnemonic):
    """Get an HGNC gene name from a UniProt mnemonic."""
    uniprot_id = uniprot_client.get_gene_name(uniprot_mnemonic)
    return uniprot_id
uniprot_to_hgnc_gene_name("CLH1_HUMAN")

'CLTC'

In [13]:
def create_hgnc_id_to_uniprot_mapping(pandas_df):
    mappings = {}
    for protein in pandas_df['Protein'].unique():
        mappings[uniprot_to_hgnc_id(protein)] = protein
    return mappings

hgnc_id_to_uniprot_mapping = create_hgnc_id_to_uniprot_mapping(pvalue_logfc_df)
hgnc_id_to_uniprot_mapping

{'1925': 'CHK1_HUMAN',
 '2092': 'CLH1_HUMAN',
 '3621': 'FCGRN_HUMAN',
 '25528': 'NECP2_HUMAN',
 '10289': 'RFA1_HUMAN'}

In [14]:
def create_hgnc_gene_name_to_uniprot_mapping(pandas_df):
    mappings = {}
    for protein in pandas_df['Protein'].unique():
        mappings[uniprot_to_hgnc_gene_name(protein)] = protein
    return mappings

hgnc_gene_name_to_uniprot_mapping = create_hgnc_gene_name_to_uniprot_mapping(pvalue_logfc_df)
hgnc_gene_name_to_uniprot_mapping

{'CHEK1': 'CHK1_HUMAN',
 'CLTC': 'CLH1_HUMAN',
 'FCGRT': 'FCGRN_HUMAN',
 'NECAP2': 'NECP2_HUMAN',
 'RPA1': 'RFA1_HUMAN'}

## STEP 3: QUERY INDRA COGEX

Using INDRA Cogex, we can extract subnetwork relationships among the proteins from the MSstats output.

In [15]:
import requests

def query_indra_subnetwork(groundings):
    """Return a list INDRA subnetwork relations based on a list of groundings."""
    res = requests.post(
        'https://discovery.indra.bio/api/indra_subnetwork_relations',
        json={'nodes': groundings}
    )
    return res.json()

In [16]:
groundings = []
for hgnc_id in hgnc_id_to_uniprot_mapping.keys():
    groundings.append(('HGNC', hgnc_id))
subnetwork_relations = query_indra_subnetwork(groundings)
subnetwork_relations[0]

{'data': {'belief': 0.98,
  'evidence_count': 1,
  'has_database_evidence': True,
  'has_reader_evidence': False,
  'has_retracted_evidence': False,
  'medscan_only': False,
  'source_counts': '{"biogrid": 1}',
  'sparser_only': False,
  'stmt_hash': 13959655286867654,
  'stmt_json': '{"type": "Complex", "members": [{"name": "RPA1", "db_refs": {"HGNC": "10289", "UP": "P27694", "EGID": "6117"}}, {"name": "CLTC", "db_refs": {"HGNC": "2092", "UP": "Q00610", "EGID": "1213"}}], "belief": 0.98, "evidence": [{"source_api": "biogrid", "pmid": "24332808", "source_id": "934345", "annotations": {"biogrid_int_id": "934345", "entrez_a": "6117", "entrez_b": "1213", "biogrid_a": "112037", "biogrid_b": "107623", "syst_name_a": null, "syst_name_b": null, "hgnc_a": "RPA1", "hgnc_b": "CLTC", "syn_a": "HSSB|MST075|REPA1|RF-A|RP-A|RPA70", "syn_b": "CHC|CHC17|CLH-17|CLTCL2|Hc", "exp_system": "Affinity Capture-MS", "exp_system_type": "physical", "author": "Marechal A (2014)", "pmid": "24332808", "organism_a"

In [ ]:
def query_indra_subnetwork(groundings):
    """Return a list INDRA subnetwork relations based on a list of groundings."""
    res = requests.post(
        'https://network.indra.bio/api/indra_subnetwork_relations',
        json={'nodes': groundings}
    )
    return res.json()

## STEP 4: ASSEMBLE DATA

INDRA has a set of assembler classes to display the data for further analysis.  Below are some examples of assemblers

### HTML Assembler

In [17]:
import json
from indra.statements import stmts_from_json

# Gather statistics for HTML presentation
unique_stmts = {entry['data']['stmt_hash']: json.loads(entry['data']['stmt_json'])
                for entry in subnetwork_relations}
ev_counts_by_hash = {entry['data']['stmt_hash']: entry['data']['evidence_count']
                     for entry in subnetwork_relations}
source_counts_by_hash = {entry['data']['stmt_hash']: json.loads(entry['data']['source_counts'])
                         for entry in subnetwork_relations}
stmts = stmts_from_json(list(unique_stmts.values()))

In [18]:
from indra.assemblers.html import HtmlAssembler
ha = HtmlAssembler(stmts,
                   title='INDRA subnetwork statements',
                   db_rest_url='https://db.indra.bio',
                   ev_counts=ev_counts_by_hash,
                   source_counts=source_counts_by_hash)
html_str = ha.make_model()

In [19]:
from IPython.core.display import HTML
HTML(html_str)

### NETWORK VISUALIZATION

We can also visualize the subnetwork acquired from INDRA Cogex using an INDRA built-in assembler

In [21]:
from indra.assemblers.graph.assembler import GraphAssembler

ga = GraphAssembler(stmts=stmts)
ga.make_model()
ga.save_pdf(file_name='graph.pdf', prog='dot')

### Tabular Format

INDRA also has an assembler for displaying INDRA cogex results as a table.

In [22]:
from indra.assemblers.indranet.assembler import IndraNetAssembler

def add_evidence_column(stmt, ev_counts = ev_counts_by_hash):
    hash = stmt.get_hash(refresh=True)
    return ev_counts[hash]

indra_net_assembler = IndraNetAssembler(statements=stmts)
relations_table = indra_net_assembler.make_df(extra_columns=[('evidence_count', add_evidence_column)])
relations_table = relations_table.sort_values(by=['evidence_count'], ascending=False)
relations_table

,agA_name,agB_name,agA_ns,agA_id,agB_ns,agB_id,residue,position,stmt_type,evidence_count,stmt_hash,belief,source_counts,initial_sign
7,CHEK1,RPA1,HGNC,1925,HGNC,10289,None,None,Phosphorylation,16,22039018332715694,0.999082,{'sparser': 1},None
5,RPA1,CHEK1,HGNC,10289,HGNC,1925,None,None,Phosphorylation,3,6298771024467860,0.923000,{'reach': 1},None
4,RPA1,CHEK1,HGNC,10289,HGNC,1925,None,None,Dephosphorylation,2,9751777159668901,0.941900,{'reach': 1},None
0,RPA1,CLTC,HGNC,10289,HGNC,2092,None,None,Complex,1,13959655286867654,0.980000,{'biogrid': 1},None
1,CLTC,RPA1,HGNC,2092,HGNC,10289,None,None,Complex,1,13959655286867654,0.980000,{'biogrid': 1},None
2,RPA1,CHEK1,HGNC,10289,HGNC,1925,S,345,Dephosphorylation,1,-4359730843333697,0.650000,{'reach': 1},None
3,RPA1,CHEK1,HGNC,10289,HGNC,1925,None,None,Activation,1,-22751136463918078,0.650000,{'reach': 1},None
6,RPA1,CHEK1,HGNC,10289,HGNC,1925,None,None,Dephosphorylation,1,-28824006899253340,0.650000,{'reach': 1},None
8,CHEK1,RPA1,HGNC,1925,HGNC,10289,None,None,Inhibition,1,-891482891587470,0.650000,{'reach': 1},None


In [27]:
from indra.assemblers.cx import CxAssembler

ca = CxAssembler(stmts, 'INDRA May Institute Network')
ncx = ca.make_model()
ca.save_model()

### Correlation Matrix

In [45]:
ABUNDANCE_PATH = "ProteinLevelData.csv" # Set this path yourself
ABUNDANCE_GROUPS_FILTER = ['PF477736']
def construct_abundance_df(filename):
    pandas_df = pd.read_csv(filename)
    pandas_df = pandas_df[pandas_df['GROUP'].isin(ABUNDANCE_GROUPS_FILTER)]
    return pandas_df

protein_abundance_df = construct_abundance_df(ABUNDANCE_PATH)
protein_abundance_df

,RUN,Protein,LogIntensities,originalRUN,GROUP,SUBJECT,TotalGroupMeasurements,NumMeasuredFeature,MissingPercentage,more50missing,NumImputedFeature
218,219,1433B_HUMAN,12.816965,230719_THP-1_Chrom_end2end_Plate1_PF477736_A07...,PF477736,7,153,10,0.0,False,0
219,220,1433B_HUMAN,12.484859,230719_THP-1_Chrom_end2end_Plate1_PF477736_C05...,PF477736,28,153,10,0.0,False,0
220,221,1433B_HUMAN,12.062958,230719_THP-1_Chrom_end2end_Plate1_PF477736_F03...,PF477736,62,153,10,0.0,False,0
221,222,1433B_HUMAN,11.954326,230719_THP-1_Chrom_end2end_Plate1_PF477736_F12...,PF477736,71,153,9,0.1,False,0
222,223,1433B_HUMAN,12.032965,230719_THP-1_Chrom_end2end_Plate1_PF477736_G12...,PF477736,83,153,10,0.0,False,0
...,...,...,...,...,...,...,...,...,...,...,...
1189787,232,ZZZ3_HUMAN,10.369426,230719_THP-1_Chrom_end2end_Plate3_VE-821_D08,PF477736,236,179,10,0.0,False,0
1189788,233,ZZZ3_HUMAN,10.587775,230719_THP-1_Chrom_end2end_Plate3_DbET6_E08,PF477736,248,179,10,0.0,False,0
1189789,234,ZZZ3_HUMAN,10.755412,230719_THP-1_Chrom_end2end_Plate3_DMSO_E10,PF477736,250,179,10,0.0,False,0
1189790,235,ZZZ3_HUMAN,10.653718,230719_THP-1_Chrom_end2end_Plate3_DMSO_F01,PF477736,253,179,10,0.0,False,0


The `dataProcess` function outputs a dataset containing protein abundances for protein and biological replicate pair.  Using the protein abundances data, we can determine the correlation of abundance between pairs of proteins.

In [46]:
import numpy as np
def calculate_correlation_matrix(pvalue_df, protein_level_summary):
    data = {}
    subjects = protein_level_summary['SUBJECT'].unique()
    for protein in pvalue_df['Protein'].unique():
        data[protein] = []
        protein_level_df = protein_level_summary[protein_level_summary['Protein'] == protein]
        for subject in subjects:
            if subject in protein_level_df['SUBJECT'].values:
                protein_level_df_subject = protein_level_df[protein_level_df['SUBJECT'] == subject]
                data[protein].append(protein_level_df_subject['LogIntensities'].iloc[0])
            else:
                data[protein].append(np.nan)
    df = pd.DataFrame(data)
    corrM = df.corr() 
    return corrM

corr_matrix = calculate_correlation_matrix(pvalue_logfc_df, protein_abundance_df)
corr_matrix

,CHK1_HUMAN,CLH1_HUMAN,FCGRN_HUMAN,NECP2_HUMAN,RFA1_HUMAN
CHK1_HUMAN,1.000000,-0.353624,-0.500364,0.418754,0.699927
CLH1_HUMAN,-0.353624,1.000000,0.613517,-0.152550,-0.489436
FCGRN_HUMAN,-0.500364,0.613517,1.000000,-0.293231,-0.498986
NECP2_HUMAN,0.418754,-0.152550,-0.293231,1.000000,0.385696
RFA1_HUMAN,0.699927,-0.489436,-0.498986,0.385696,1.000000


In [47]:
LOG_FC_FILTER = 0.25
pvalue_logfc_df['log2FC'] = pvalue_logfc_df['log2FC'].astype(float)
logfc_proteins = pvalue_logfc_df[pvalue_logfc_df['log2FC'].abs() > LOG_FC_FILTER]
logfc_proteins

,Protein,Label,log2FC,SE,Tvalue,DF,pvalue,adj.pvalue,issue,MissingPercentage,ImputationPercentage
32541,CLH1_HUMAN,DMSO-PF477736,-0.269784,0.056036,-4.814489,260.0,2.506764e-06,3.094600e-03,NaN,0.143333,0.0
63546,FCGRN_HUMAN,DMSO-PF477736,-0.674851,0.132991,-5.074427,179.0,9.674775e-07,1.592468e-03,NaN,0.195918,0.0
115656,NECP2_HUMAN,DMSO-PF477736,0.693701,0.097077,7.145875,259.0,9.076739e-12,2.241047e-08,NaN,0.040000,0.0
152151,RFA1_HUMAN,DMSO-PF477736,-0.603824,0.076190,-7.925239,260.0,6.705747e-14,3.311298e-10,NaN,0.120667,0.0


We can get an explanation from analyzing the INDRA statements.  The drug is supposed to inhibit CHEK1 / decrease CHEK1.  Because there is less CHEK1 influence, RPA1 increases in abundance since CHEK1 inhibits RPA1, and then that causes CLTC to go up too.  RPA1 activates CHEK1